In [1]:
from kan import *
import aeon
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import numpy as np
from aeon.datasets import load_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from kan.utils import create_dataset_from_data
import torch

dtype = torch.get_default_dtype()

# Dataset

In [3]:
X, y = load_classification(name="HandOutlines")
y = np.array(list(map(int, y)))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#X, y = load_classification(name="Earthquakes")
X, y = load_classification(name="HandOutlines")
X = torch.tensor(X.squeeze(1)).type(dtype)
y = np.array(y_test, dtype=int)
# y = torch.tensor((list(map(int, y)))).type(torch.long)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(X.shape, y.shape, device)

torch.Size([1370, 2709]) torch.Size([1370]) cuda


## Data split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) 
dataset = create_dataset_from_data(X_train, y_train, device=device)
n_inputs, input_dim = dataset['train_input'].shape

# Baselines

# KAN

In [11]:
# [n_dim, 30, 30, 30, n_class]
# [n_dim, 30, 30, 30, 30, n_class]

# [n_dim, 3, 3, n_class] k=3, g-5
# [n_dim, 3, 3, 3, n_class] k=3, g-5

In [15]:
model = KAN(width=[input_dim, 3, 3, 2], grid=3, k=3, seed=2024, device=device)
model(dataset['train_input'])
#model.plot(beta=10)

checkpoint directory created: ./model
saving model version 0.0


tensor([[ 0.0183, -0.0122],
        [ 0.0175, -0.0134],
        [ 0.0176, -0.0087],
        ...,
        [ 0.0193, -0.0129],
        [ 0.0217, -0.0143],
        [ 0.0199, -0.0151]], device='cuda:0', grad_fn=<AddBackward0>)

In [17]:
def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).type(dtype))

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).type(dtype))

In [18]:
results = model.fit(dataset, opt="LBFGS", steps=50, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss());
results['train_acc'][-1], results['test_acc'][-1]

| train_loss: 3.00e-01 | test_loss: 9.28e-01 | reg: 3.33e+02 | : 100%|█| 50/50 [00:52<00:00,  1.05s/

saving model version 0.1


(0.9716024398803711, 0.8785425424575806)